In [1]:
from tqdm import tqdm
import json
import requests
import geopandas as gpd
import geocoder
import pandas as pd
import numpy as np
from shapely.geometry import LineString, Point, Polygon
from func import *

In [2]:
specified_order = [
    '北投區', '士林區', '內湖區', '南港區',
    '松山區', '信義區', '中山區', '大同區',
    '中正區', '萬華區', '大安區', '文山區'
]

In [3]:
breastfeeding_original = get_datataipei_api('4350117f-8697-4903-8a49-a7a59f30eeb1')
breastfeeding_copy = breastfeeding_original.copy()
breastfeeding_copy

,_id,_importdate,行政區,機構名稱,地址,電話,開放時間,位置指引,基本設備,友善設備或服務,優良哺集乳室認證效期,輪椅使用,貼心小提醒
0,1,"{'date': '2023-03-22 11:30:34.622182', 'timezo...",松山區,臺北市松山區健康服務中心,臺北市松山區八德路4段692號2樓,02-27671757轉6062,週一至週五08:30~17:30,松山區行政大樓2樓服務台旁,靠背椅;有蓋垃圾桶;電源設備 （插座或延長線）;可由內部上鎖之門;緊急求救設備 （求救鈴或電...,飲水服務;尿布台;母乳專用冰箱;消毒器具;置物空間;嬰兒車停放空間;母乳哺育宣導資訊;設置意...,無,可提供輪椅媽媽使用,自行前往哺集乳室，並登記；哺集乳室資訊依各場所實際公告內容為主
1,2,"{'date': '2023-03-22 11:30:34.631819', 'timezo...",松山區,內政部營建署,臺北市松山區八德路2段342號1樓,02-87712437,週一至週五08:00~17:00,101室,靠背椅;有蓋垃圾桶;電源設備 （插座或延長線）;可由內部上鎖之門;緊急求救設備 （求救鈴或電...,飲水服務;尿布台;母乳專用冰箱;免費衛生用品;置物空間;母乳哺育宣導資訊;設置意見回饋箱,112年8月,無,使用者請於開放時間向本署秘書室事務課洽借鑰匙，並填寫哺(集)乳室鑰匙借用登記表、使用登記統計...
2,3,"{'date': '2023-03-22 11:30:34.633594', 'timezo...",松山區,交通部運輸研究所,臺北市松山區敦化北路240號,02-23496734轉6733,每日24小時開放,地下1樓國際會議廳後方,靠背椅;有蓋垃圾桶;電源設備 （插座或延長線）;可由內部上鎖之門;緊急求救設備 （求救鈴或電...,飲水服務;母乳專用冰箱;置物空間;嬰兒車停放空間;母乳哺育宣導資訊,無,可提供輪椅媽媽使用,哺集乳室資訊依各場所實際公告內容為主
3,4,"{'date': '2023-03-22 11:30:34.635314', 'timezo...",松山區,交通部公路總局臺北市區監理所,臺北市松山區八德路4段21號,02-27630155,週一至週五08:30~18:00,2樓電梯旁,靠背椅;有蓋垃圾桶;電源設備 （插座或延長線）;可由內部上鎖之門;緊急求救設備 （求救鈴或電...,尿布台;母乳專用冰箱;置物空間;嬰兒車停放空間;母乳哺育宣導資訊;進行滿意度調查,112年8月,可提供輪椅媽媽使用,自行前往哺集乳室；哺集乳室資訊依各場所實際公告內容為主
4,5,"{'date': '2023-03-22 11:30:34.637038', 'timezo...",松山區,臺北自來水事業處東區營業分處,臺北市松山區光復北路266號,02-87703362,週一至週五08:30~17:30,1樓3號服務櫃台正對面,靠背椅;有蓋垃圾桶;電源設備 （插座或延長線）;可由內部上鎖之門;緊急求救設備 （求救鈴或電...,飲水服務;尿布台;母乳專用冰箱;置物空間;嬰兒車停放空間;母乳哺育宣導資訊;進行滿意度調查,112年8月,無,自行前往哺集乳室，並登記；哺集乳室資訊依各場所實際公告內容為主
...,...,...,...,...,...,...,...,...,...,...,...,...,...
533,534,"{'date': '2023-03-22 11:30:35.447849', 'timezo...",北投區,臺北市政府消防局第四大隊陽明山分隊,臺北市北投區勝利路1號,02-28616119,每日24小時開放,1樓值班室旁,靠背椅;有蓋垃圾桶;電源設備 （插座或延長線）;可由內部上鎖之門;緊急求救設備 （求救鈴或電...,飲水服務;尿布台;置物空間;母乳哺育宣導資訊,112年8月,可提供輪椅媽媽使用,自行前往哺集乳室，並登記；哺集乳室資訊依各場所實際公告內容為主
534,535,"{'date': '2023-03-22 11:30:35.449273', 'timezo...",北投區,臺北市政府環境保護局北投垃圾焚化廠,臺北市北投區洲美街271號,02-28360050,週一至週五09:00~17:00,1樓主題館,靠背椅;有蓋垃圾桶;電源設備 （插座或延長線）;可由內部上鎖之門;緊急求救設備 （求救鈴或電...,以室內設備為主,無,以室內設備為主,自行前往哺集乳室，並登記；哺集乳室資訊依各場所實際公告內容為主
535,536,"{'date': '2023-03-22 11:30:35.450817', 'timezo...",北投區,陽明山國家公園管理處陽明書屋,臺北市北投區中興路12號,02-28611444,週一至週五09:00~17:00,遊客中心,靠背椅;有蓋垃圾桶;電源設備 （插座或延長線）;可由內部上鎖之門;緊急求救設備 （求救鈴或電...,飲水服務;尿布台;母乳專用冰箱;免費衛生用品;置物空間;嬰兒車停放空間;母乳哺育宣導資訊;設...,112年8月,可提供輪椅媽媽使用,自行前往哺集乳室，並登記；哺集乳室資訊依各場所實際公告內容為主
536,537,"{'date': '2023-03-22 11:30:35.452159', 'timezo...",北投區,家樂福北投店,臺北市北投區中和街366號,0963-079728,週一至週日08:00~23:00,地下1樓,靠背椅;有蓋垃圾桶;電源設備 （插座或延長線）;可由內部上鎖之門;緊急求救設備 （求救鈴或電...,以室內設備為主,無,以室內設備為主,自行前往哺集乳室，並登記；哺集乳室資訊依各場所實際公告內容為主


In [4]:
breastfeeding_chart = breastfeeding_copy["行政區"]
breastfeeding_chart

0      松山區
1      松山區
2      松山區
3      松山區
4      松山區
      ... 
533    北投區
534    北投區
535    北投區
536    北投區
537    北投區
Name: 行政區, Length: 538, dtype: object

In [5]:
breastfeeding_count = breastfeeding_chart.value_counts()

In [6]:
result = {
  "data": [
    {
      "name": "breastfeeding_room_count",
      "data": [
        { "x": dist, "y": int(breastfeeding_count[dist]) }
        for dist in specified_order
      ]
    }
  ]
}
result

{'data': [{'name': 'breastfeeding_room_count',
   'data': [{'x': '北投區', 'y': 41},
    {'x': '士林區', 'y': 47},
    {'x': '內湖區', 'y': 24},
    {'x': '南港區', 'y': 34},
    {'x': '松山區', 'y': 47},
    {'x': '信義區', 'y': 53},
    {'x': '中山區', 'y': 46},
    {'x': '大同區', 'y': 34},
    {'x': '中正區', 'y': 87},
    {'x': '萬華區', 'y': 36},
    {'x': '大安區', 'y': 41},
    {'x': '文山區', 'y': 48}]}]}

In [7]:
with open('4003.json', "w", encoding="utf-8") as json_file:
    json.dump(result, json_file, ensure_ascii=False, indent=4)

In [8]:
breastfeeding_map = breastfeeding_copy[["行政區", "機構名稱", "地址", "電話", "開放時間", "位置指引", "友善設備或服務", "輪椅使用", "貼心小提醒"]]
breastfeeding_map

,行政區,機構名稱,地址,電話,開放時間,位置指引,友善設備或服務,輪椅使用,貼心小提醒
0,松山區,臺北市松山區健康服務中心,臺北市松山區八德路4段692號2樓,02-27671757轉6062,週一至週五08:30~17:30,松山區行政大樓2樓服務台旁,飲水服務;尿布台;母乳專用冰箱;消毒器具;置物空間;嬰兒車停放空間;母乳哺育宣導資訊;設置意...,可提供輪椅媽媽使用,自行前往哺集乳室，並登記；哺集乳室資訊依各場所實際公告內容為主
1,松山區,內政部營建署,臺北市松山區八德路2段342號1樓,02-87712437,週一至週五08:00~17:00,101室,飲水服務;尿布台;母乳專用冰箱;免費衛生用品;置物空間;母乳哺育宣導資訊;設置意見回饋箱,無,使用者請於開放時間向本署秘書室事務課洽借鑰匙，並填寫哺(集)乳室鑰匙借用登記表、使用登記統計...
2,松山區,交通部運輸研究所,臺北市松山區敦化北路240號,02-23496734轉6733,每日24小時開放,地下1樓國際會議廳後方,飲水服務;母乳專用冰箱;置物空間;嬰兒車停放空間;母乳哺育宣導資訊,可提供輪椅媽媽使用,哺集乳室資訊依各場所實際公告內容為主
3,松山區,交通部公路總局臺北市區監理所,臺北市松山區八德路4段21號,02-27630155,週一至週五08:30~18:00,2樓電梯旁,尿布台;母乳專用冰箱;置物空間;嬰兒車停放空間;母乳哺育宣導資訊;進行滿意度調查,可提供輪椅媽媽使用,自行前往哺集乳室；哺集乳室資訊依各場所實際公告內容為主
4,松山區,臺北自來水事業處東區營業分處,臺北市松山區光復北路266號,02-87703362,週一至週五08:30~17:30,1樓3號服務櫃台正對面,飲水服務;尿布台;母乳專用冰箱;置物空間;嬰兒車停放空間;母乳哺育宣導資訊;進行滿意度調查,無,自行前往哺集乳室，並登記；哺集乳室資訊依各場所實際公告內容為主
...,...,...,...,...,...,...,...,...,...
533,北投區,臺北市政府消防局第四大隊陽明山分隊,臺北市北投區勝利路1號,02-28616119,每日24小時開放,1樓值班室旁,飲水服務;尿布台;置物空間;母乳哺育宣導資訊,可提供輪椅媽媽使用,自行前往哺集乳室，並登記；哺集乳室資訊依各場所實際公告內容為主
534,北投區,臺北市政府環境保護局北投垃圾焚化廠,臺北市北投區洲美街271號,02-28360050,週一至週五09:00~17:00,1樓主題館,以室內設備為主,以室內設備為主,自行前往哺集乳室，並登記；哺集乳室資訊依各場所實際公告內容為主
535,北投區,陽明山國家公園管理處陽明書屋,臺北市北投區中興路12號,02-28611444,週一至週五09:00~17:00,遊客中心,飲水服務;尿布台;母乳專用冰箱;免費衛生用品;置物空間;嬰兒車停放空間;母乳哺育宣導資訊;設...,可提供輪椅媽媽使用,自行前往哺集乳室，並登記；哺集乳室資訊依各場所實際公告內容為主
536,北投區,家樂福北投店,臺北市北投區中和街366號,0963-079728,週一至週日08:00~23:00,地下1樓,以室內設備為主,以室內設備為主,自行前往哺集乳室，並登記；哺集乳室資訊依各場所實際公告內容為主


In [9]:
def getCoordinate(addr):
  addr_info = geocoder.arcgis(addr)
  addr_json = addr_info.json
  # print(addr_json)
  try:
    if(addr_json != None):
      # print(addr_json['lng'])
      return Point(
        addr_json['lng'],
        addr_json['lat']
    )
    return 'NA'
  except:
    return 'NA'

In [42]:
breastfeeding_map["geometry"] = breastfeeding_map["地址"].apply(
    getCoordinate
)

C:\Users\Joachim Chen\AppData\Local\Temp\ipykernel_13948\3008916829.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  breastfeeding_map["geometry"] = breastfeeding_map["地址"].apply(


In [10]:

breastfeeding_map["address"] = breastfeeding_map["地址"]
breastfeeding_map["contact"] = breastfeeding_map["電話"]
breastfeeding_map["opening_hours"] = breastfeeding_map["開放時間"]
breastfeeding_map["dist"] = breastfeeding_map["行政區"]
breastfeeding_map["reminder"] = breastfeeding_map["貼心小提醒"]
breastfeeding_map["name"] = breastfeeding_map["機構名稱"]
breastfeeding_map["guide"] = breastfeeding_map["位置指引"]
# breastfeeding_map.drop(columns=["地址", "電話", "開放時間", "行政區", "貼心小提醒"], inplace=True)

C:\Users\Joachim Chen\AppData\Local\Temp\ipykernel_25016\2747489199.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  breastfeeding_map["address"] = breastfeeding_map["地址"]
C:\Users\Joachim Chen\AppData\Local\Temp\ipykernel_25016\2747489199.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  breastfeeding_map["contact"] = breastfeeding_map["電話"]
C:\Users\Joachim Chen\AppData\Local\Temp\ipykernel_25016\2747489199.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFra

In [11]:
breastfeeding_map

,行政區,機構名稱,地址,電話,開放時間,位置指引,友善設備或服務,輪椅使用,貼心小提醒,address,contact,opening_hours,dist,reminder,name,guide
0,松山區,臺北市松山區健康服務中心,臺北市松山區八德路4段692號2樓,02-27671757轉6062,週一至週五08:30~17:30,松山區行政大樓2樓服務台旁,飲水服務;尿布台;母乳專用冰箱;消毒器具;置物空間;嬰兒車停放空間;母乳哺育宣導資訊;設置意...,可提供輪椅媽媽使用,自行前往哺集乳室，並登記；哺集乳室資訊依各場所實際公告內容為主,臺北市松山區八德路4段692號2樓,02-27671757轉6062,週一至週五08:30~17:30,松山區,自行前往哺集乳室，並登記；哺集乳室資訊依各場所實際公告內容為主,臺北市松山區健康服務中心,松山區行政大樓2樓服務台旁
1,松山區,內政部營建署,臺北市松山區八德路2段342號1樓,02-87712437,週一至週五08:00~17:00,101室,飲水服務;尿布台;母乳專用冰箱;免費衛生用品;置物空間;母乳哺育宣導資訊;設置意見回饋箱,無,使用者請於開放時間向本署秘書室事務課洽借鑰匙，並填寫哺(集)乳室鑰匙借用登記表、使用登記統計...,臺北市松山區八德路2段342號1樓,02-87712437,週一至週五08:00~17:00,松山區,使用者請於開放時間向本署秘書室事務課洽借鑰匙，並填寫哺(集)乳室鑰匙借用登記表、使用登記統計...,內政部營建署,101室
2,松山區,交通部運輸研究所,臺北市松山區敦化北路240號,02-23496734轉6733,每日24小時開放,地下1樓國際會議廳後方,飲水服務;母乳專用冰箱;置物空間;嬰兒車停放空間;母乳哺育宣導資訊,可提供輪椅媽媽使用,哺集乳室資訊依各場所實際公告內容為主,臺北市松山區敦化北路240號,02-23496734轉6733,每日24小時開放,松山區,哺集乳室資訊依各場所實際公告內容為主,交通部運輸研究所,地下1樓國際會議廳後方
3,松山區,交通部公路總局臺北市區監理所,臺北市松山區八德路4段21號,02-27630155,週一至週五08:30~18:00,2樓電梯旁,尿布台;母乳專用冰箱;置物空間;嬰兒車停放空間;母乳哺育宣導資訊;進行滿意度調查,可提供輪椅媽媽使用,自行前往哺集乳室；哺集乳室資訊依各場所實際公告內容為主,臺北市松山區八德路4段21號,02-27630155,週一至週五08:30~18:00,松山區,自行前往哺集乳室；哺集乳室資訊依各場所實際公告內容為主,交通部公路總局臺北市區監理所,2樓電梯旁
4,松山區,臺北自來水事業處東區營業分處,臺北市松山區光復北路266號,02-87703362,週一至週五08:30~17:30,1樓3號服務櫃台正對面,飲水服務;尿布台;母乳專用冰箱;置物空間;嬰兒車停放空間;母乳哺育宣導資訊;進行滿意度調查,無,自行前往哺集乳室，並登記；哺集乳室資訊依各場所實際公告內容為主,臺北市松山區光復北路266號,02-87703362,週一至週五08:30~17:30,松山區,自行前往哺集乳室，並登記；哺集乳室資訊依各場所實際公告內容為主,臺北自來水事業處東區營業分處,1樓3號服務櫃台正對面
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
533,北投區,臺北市政府消防局第四大隊陽明山分隊,臺北市北投區勝利路1號,02-28616119,每日24小時開放,1樓值班室旁,飲水服務;尿布台;置物空間;母乳哺育宣導資訊,可提供輪椅媽媽使用,自行前往哺集乳室，並登記；哺集乳室資訊依各場所實際公告內容為主,臺北市北投區勝利路1號,02-28616119,每日24小時開放,北投區,自行前往哺集乳室，並登記；哺集乳室資訊依各場所實際公告內容為主,臺北市政府消防局第四大隊陽明山分隊,1樓值班室旁
534,北投區,臺北市政府環境保護局北投垃圾焚化廠,臺北市北投區洲美街271號,02-28360050,週一至週五09:00~17:00,1樓主題館,以室內設備為主,以室內設備為主,自行前往哺集乳室，並登記；哺集乳室資訊依各場所實際公告內容為主,臺北市北投區洲美街271號,02-28360050,週一至週五09:00~17:00,北投區,自行前往哺集乳室，並登記；哺集乳室資訊依各場所實際公告內容為主,臺北市政府環境保護局北投垃圾焚化廠,1樓主題館
535,北投區,陽明山國家公園管理處陽明書屋,臺北市北投區中興路12號,02-28611444,週一至週五09:00~17:00,遊客中心,飲水服務;尿布台;母乳專用冰箱;免費衛生用品;置物空間;嬰兒車停放空間;母乳哺育宣導資訊;設...,可提供輪椅媽媽使用,自行前往哺集乳室，並登記；哺集乳室資訊依各場所實際公告內容為主,臺北市北投區中興路12號,02-28611444,週一至週五09:00~17:00,北投區,自行前往哺集乳室，並登記；哺集乳室資訊依各場所實際公告內容為主,陽明山國家公園管理處陽明書屋,遊客中心
536,北投區,家樂福北投店,臺北市北投區中和街366號,0963-079728,週一至週日08:00~23:00,地下1樓,以室內設備為主,以室內設備為主,自行前往哺集乳室，並登記；哺集乳室資訊依各場所實際公告內容為主,臺北市北投區中和街366號,0963-079728,週一至週日08:00~23:00,北投區,自行前往哺集乳室，並登記；哺集乳室資訊依各場所實際公告內容為主,家樂福北投店,地下1樓


In [46]:
breastfeeding_map = breastfeeding_map[breastfeeding_map["geometry"] != "NA"]
breastfeeding_map

,行政區,機構名稱,地址,電話,開放時間,位置指引,友善設備或服務,輪椅使用,貼心小提醒,geometry,address,contact,opening_hours,dist,reminder,name,guide
0,松山區,臺北市松山區健康服務中心,臺北市松山區八德路4段692號2樓,02-27671757轉6062,週一至週五08:30~17:30,松山區行政大樓2樓服務台旁,飲水服務;尿布台;母乳專用冰箱;消毒器具;置物空間;嬰兒車停放空間;母乳哺育宣導資訊;設置意...,可提供輪椅媽媽使用,自行前往哺集乳室，並登記；哺集乳室資訊依各場所實際公告內容為主,POINT (121.5771700262917 25.049850016389897),臺北市松山區八德路4段692號2樓,02-27671757轉6062,週一至週五08:30~17:30,松山區,自行前往哺集乳室，並登記；哺集乳室資訊依各場所實際公告內容為主,臺北市松山區健康服務中心,松山區行政大樓2樓服務台旁
1,松山區,內政部營建署,臺北市松山區八德路2段342號1樓,02-87712437,週一至週五08:00~17:00,101室,飲水服務;尿布台;母乳專用冰箱;免費衛生用品;置物空間;母乳哺育宣導資訊;設置意見回饋箱,無,使用者請於開放時間向本署秘書室事務課洽借鑰匙，並填寫哺(集)乳室鑰匙借用登記表、使用登記統計...,POINT (121.54448001717424 25.047590003836703),臺北市松山區八德路2段342號1樓,02-87712437,週一至週五08:00~17:00,松山區,使用者請於開放時間向本署秘書室事務課洽借鑰匙，並填寫哺(集)乳室鑰匙借用登記表、使用登記統計...,內政部營建署,101室
2,松山區,交通部運輸研究所,臺北市松山區敦化北路240號,02-23496734轉6733,每日24小時開放,地下1樓國際會議廳後方,飲水服務;母乳專用冰箱;置物空間;嬰兒車停放空間;母乳哺育宣導資訊,可提供輪椅媽媽使用,哺集乳室資訊依各場所實際公告內容為主,POINT (121.54948996452123 25.060069987112684),臺北市松山區敦化北路240號,02-23496734轉6733,每日24小時開放,松山區,哺集乳室資訊依各場所實際公告內容為主,交通部運輸研究所,地下1樓國際會議廳後方
3,松山區,交通部公路總局臺北市區監理所,臺北市松山區八德路4段21號,02-27630155,週一至週五08:30~18:00,2樓電梯旁,尿布台;母乳專用冰箱;置物空間;嬰兒車停放空間;母乳哺育宣導資訊;進行滿意度調查,可提供輪椅媽媽使用,自行前往哺集乳室；哺集乳室資訊依各場所實際公告內容為主,POINT (121.55902002079364 25.048449987102515),臺北市松山區八德路4段21號,02-27630155,週一至週五08:30~18:00,松山區,自行前往哺集乳室；哺集乳室資訊依各場所實際公告內容為主,交通部公路總局臺北市區監理所,2樓電梯旁
4,松山區,臺北自來水事業處東區營業分處,臺北市松山區光復北路266號,02-87703362,週一至週五08:30~17:30,1樓3號服務櫃台正對面,飲水服務;尿布台;母乳專用冰箱;置物空間;嬰兒車停放空間;母乳哺育宣導資訊;進行滿意度調查,無,自行前往哺集乳室，並登記；哺集乳室資訊依各場所實際公告內容為主,POINT (121.55469998789704 25.059179996633517),臺北市松山區光復北路266號,02-87703362,週一至週五08:30~17:30,松山區,自行前往哺集乳室，並登記；哺集乳室資訊依各場所實際公告內容為主,臺北自來水事業處東區營業分處,1樓3號服務櫃台正對面
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
533,北投區,臺北市政府消防局第四大隊陽明山分隊,臺北市北投區勝利路1號,02-28616119,每日24小時開放,1樓值班室旁,飲水服務;尿布台;置物空間;母乳哺育宣導資訊,可提供輪椅媽媽使用,自行前往哺集乳室，並登記；哺集乳室資訊依各場所實際公告內容為主,POINT (121.50253000000009 25.13289000000003),臺北市北投區勝利路1號,02-28616119,每日24小時開放,北投區,自行前往哺集乳室，並登記；哺集乳室資訊依各場所實際公告內容為主,臺北市政府消防局第四大隊陽明山分隊,1樓值班室旁
534,北投區,臺北市政府環境保護局北投垃圾焚化廠,臺北市北投區洲美街271號,02-28360050,週一至週五09:00~17:00,1樓主題館,以室內設備為主,以室內設備為主,自行前往哺集乳室，並登記；哺集乳室資訊依各場所實際公告內容為主,POINT (121.49870996433248 25.109180018822286),臺北市北投區洲美街271號,02-28360050,週一至週五09:00~17:00,北投區,自行前往哺集乳室，並登記；哺集乳室資訊依各場所實際公告內容為主,臺北市政府環境保護局北投垃圾焚化廠,1樓主題館
535,北投區,陽明山國家公園管理處陽明書屋,臺北市北投區中興路12號,02-28611444,週一至週五09:00~17:00,遊客中心,飲水服務;尿布台;母乳專用冰箱;免費衛生用品;置物空間;嬰兒車停放空間;母乳哺育宣導資訊;設...,可提供輪椅媽媽使用,自行前往哺集乳室，並登記；哺集乳室資訊依各場所實際公告內容為主,POINT (121.53941139829493 25.16162818548129),臺北市北投區中興路12號,02-28611444,週一至週五09:00~17:00,北投區,自行前往哺集乳室，並登記；哺集乳室資訊依各場所實際公告內容為主,陽明山國家公園管理處陽明書屋,遊客中心
536,北投區,家樂福北投店,臺北市北投區中和街366號,0963-079728,週一至週日08:00~23:00,地下1樓,以室內設備為主,以室內設備為主,自行前往哺集乳室，並登記；哺集乳室資訊依各場所實際公告內容為主,POINT (121.49945000256383 25.14168998465952),臺北市北投區中和街366號,0963-079728,週一至週日08:00~23:00,北投區,自行前往哺集乳室，並登記；哺集乳室資訊依各場所實際公告內容為主,家樂福北投店,地下1樓


In [12]:
breastfeeding_map.index.to_series()

0        0
1        1
2        2
3        3
4        4
      ... 
533    533
534    534
535    535
536    536
537    537
Length: 538, dtype: int64

In [13]:
def getServices(idx):
    if (breastfeeding_map.at[idx, "輪椅使用"] == "以室內設備為主" or breastfeeding_map.at[idx, "友善設備或服務"] == "以室內設備為主"):
        return ["無資料"]
    else:
        room_services = []
        services_from_str = breastfeeding_map.at[idx, "友善設備或服務"].split(';')
        services_list = ["飲水服務", "尿布台", "母乳專用冰箱", "免費衛生用品", "置物空間", "嬰兒車停放空間"]
        if(breastfeeding_map.at[idx, "輪椅使用"] != "無"):
            room_services.append("輪椅")
        for service in services_list:
            if service in services_from_str:
                room_services.append(service)
        return room_services
getServices(4)         

['飲水服務', '尿布台', '母乳專用冰箱', '置物空間', '嬰兒車停放空間']

In [14]:
breastfeeding_map["features"] = breastfeeding_map.index.to_series().apply(
    getServices
)
breastfeeding_map

,行政區,機構名稱,地址,電話,開放時間,位置指引,友善設備或服務,輪椅使用,貼心小提醒,address,contact,opening_hours,dist,reminder,name,guide,features
0,松山區,臺北市松山區健康服務中心,臺北市松山區八德路4段692號2樓,02-27671757轉6062,週一至週五08:30~17:30,松山區行政大樓2樓服務台旁,飲水服務;尿布台;母乳專用冰箱;消毒器具;置物空間;嬰兒車停放空間;母乳哺育宣導資訊;設置意...,可提供輪椅媽媽使用,自行前往哺集乳室，並登記；哺集乳室資訊依各場所實際公告內容為主,臺北市松山區八德路4段692號2樓,02-27671757轉6062,週一至週五08:30~17:30,松山區,自行前往哺集乳室，並登記；哺集乳室資訊依各場所實際公告內容為主,臺北市松山區健康服務中心,松山區行政大樓2樓服務台旁,"[輪椅, 飲水服務, 尿布台, 母乳專用冰箱, 置物空間, 嬰兒車停放空間]"
1,松山區,內政部營建署,臺北市松山區八德路2段342號1樓,02-87712437,週一至週五08:00~17:00,101室,飲水服務;尿布台;母乳專用冰箱;免費衛生用品;置物空間;母乳哺育宣導資訊;設置意見回饋箱,無,使用者請於開放時間向本署秘書室事務課洽借鑰匙，並填寫哺(集)乳室鑰匙借用登記表、使用登記統計...,臺北市松山區八德路2段342號1樓,02-87712437,週一至週五08:00~17:00,松山區,使用者請於開放時間向本署秘書室事務課洽借鑰匙，並填寫哺(集)乳室鑰匙借用登記表、使用登記統計...,內政部營建署,101室,"[飲水服務, 尿布台, 母乳專用冰箱, 免費衛生用品, 置物空間]"
2,松山區,交通部運輸研究所,臺北市松山區敦化北路240號,02-23496734轉6733,每日24小時開放,地下1樓國際會議廳後方,飲水服務;母乳專用冰箱;置物空間;嬰兒車停放空間;母乳哺育宣導資訊,可提供輪椅媽媽使用,哺集乳室資訊依各場所實際公告內容為主,臺北市松山區敦化北路240號,02-23496734轉6733,每日24小時開放,松山區,哺集乳室資訊依各場所實際公告內容為主,交通部運輸研究所,地下1樓國際會議廳後方,"[輪椅, 飲水服務, 母乳專用冰箱, 置物空間, 嬰兒車停放空間]"
3,松山區,交通部公路總局臺北市區監理所,臺北市松山區八德路4段21號,02-27630155,週一至週五08:30~18:00,2樓電梯旁,尿布台;母乳專用冰箱;置物空間;嬰兒車停放空間;母乳哺育宣導資訊;進行滿意度調查,可提供輪椅媽媽使用,自行前往哺集乳室；哺集乳室資訊依各場所實際公告內容為主,臺北市松山區八德路4段21號,02-27630155,週一至週五08:30~18:00,松山區,自行前往哺集乳室；哺集乳室資訊依各場所實際公告內容為主,交通部公路總局臺北市區監理所,2樓電梯旁,"[輪椅, 尿布台, 母乳專用冰箱, 置物空間, 嬰兒車停放空間]"
4,松山區,臺北自來水事業處東區營業分處,臺北市松山區光復北路266號,02-87703362,週一至週五08:30~17:30,1樓3號服務櫃台正對面,飲水服務;尿布台;母乳專用冰箱;置物空間;嬰兒車停放空間;母乳哺育宣導資訊;進行滿意度調查,無,自行前往哺集乳室，並登記；哺集乳室資訊依各場所實際公告內容為主,臺北市松山區光復北路266號,02-87703362,週一至週五08:30~17:30,松山區,自行前往哺集乳室，並登記；哺集乳室資訊依各場所實際公告內容為主,臺北自來水事業處東區營業分處,1樓3號服務櫃台正對面,"[飲水服務, 尿布台, 母乳專用冰箱, 置物空間, 嬰兒車停放空間]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
533,北投區,臺北市政府消防局第四大隊陽明山分隊,臺北市北投區勝利路1號,02-28616119,每日24小時開放,1樓值班室旁,飲水服務;尿布台;置物空間;母乳哺育宣導資訊,可提供輪椅媽媽使用,自行前往哺集乳室，並登記；哺集乳室資訊依各場所實際公告內容為主,臺北市北投區勝利路1號,02-28616119,每日24小時開放,北投區,自行前往哺集乳室，並登記；哺集乳室資訊依各場所實際公告內容為主,臺北市政府消防局第四大隊陽明山分隊,1樓值班室旁,"[輪椅, 飲水服務, 尿布台, 置物空間]"
534,北投區,臺北市政府環境保護局北投垃圾焚化廠,臺北市北投區洲美街271號,02-28360050,週一至週五09:00~17:00,1樓主題館,以室內設備為主,以室內設備為主,自行前往哺集乳室，並登記；哺集乳室資訊依各場所實際公告內容為主,臺北市北投區洲美街271號,02-28360050,週一至週五09:00~17:00,北投區,自行前往哺集乳室，並登記；哺集乳室資訊依各場所實際公告內容為主,臺北市政府環境保護局北投垃圾焚化廠,1樓主題館,[無資料]
535,北投區,陽明山國家公園管理處陽明書屋,臺北市北投區中興路12號,02-28611444,週一至週五09:00~17:00,遊客中心,飲水服務;尿布台;母乳專用冰箱;免費衛生用品;置物空間;嬰兒車停放空間;母乳哺育宣導資訊;設...,可提供輪椅媽媽使用,自行前往哺集乳室，並登記；哺集乳室資訊依各場所實際公告內容為主,臺北市北投區中興路12號,02-28611444,週一至週五09:00~17:00,北投區,自行前往哺集乳室，並登記；哺集乳室資訊依各場所實際公告內容為主,陽明山國家公園管理處陽明書屋,遊客中心,"[輪椅, 飲水服務, 尿布台, 母乳專用冰箱, 免費衛生用品, 置物空間, 嬰兒車停放空間]"
536,北投區,家樂福北投店,臺北市北投區中和街366號,0963-079728,週一至週日08:00~23:00,地下1樓,以室內設備為主,以室內設備為主,自行前往哺集乳室，並登記；哺集乳室資訊依各場所實際公告內容為主,臺北市北投區中和街366號,0963-079728,週一至週日08:00~23:00,北投區,自行前往哺集乳室，並登記；哺集乳室資訊依各場所實際公告內容為主,家樂福北投店,地下1樓,[無資料]


In [35]:
services_list = ["輪椅","飲水服務", "尿布台", "母乳專用冰箱", "免費衛生用品", "置物空間", "嬰兒車停放空間"]
services_dict = dict.fromkeys(services_list, 0)
total = 0
for i in range(len(breastfeeding_map["features"])):
    for service in breastfeeding_map.loc[i, "features"]:
        if(service in services_dict):
            services_dict[service] += 1
services_dict

{'輪椅': 280,
 '飲水服務': 272,
 '尿布台': 386,
 '母乳專用冰箱': 234,
 '免費衛生用品': 234,
 '置物空間': 375,
 '嬰兒車停放空間': 294}

In [54]:
breastfeeding_map.drop(columns=["地址", "電話", "開放時間", "行政區", "貼心小提醒", "機構名稱", "友善設備或服務", "輪椅使用","位置指引"], inplace=True)

In [15]:
breastfeeding_map

,行政區,機構名稱,地址,電話,開放時間,位置指引,友善設備或服務,輪椅使用,貼心小提醒,address,contact,opening_hours,dist,reminder,name,guide,features
0,松山區,臺北市松山區健康服務中心,臺北市松山區八德路4段692號2樓,02-27671757轉6062,週一至週五08:30~17:30,松山區行政大樓2樓服務台旁,飲水服務;尿布台;母乳專用冰箱;消毒器具;置物空間;嬰兒車停放空間;母乳哺育宣導資訊;設置意...,可提供輪椅媽媽使用,自行前往哺集乳室，並登記；哺集乳室資訊依各場所實際公告內容為主,臺北市松山區八德路4段692號2樓,02-27671757轉6062,週一至週五08:30~17:30,松山區,自行前往哺集乳室，並登記；哺集乳室資訊依各場所實際公告內容為主,臺北市松山區健康服務中心,松山區行政大樓2樓服務台旁,"[輪椅, 飲水服務, 尿布台, 母乳專用冰箱, 置物空間, 嬰兒車停放空間]"
1,松山區,內政部營建署,臺北市松山區八德路2段342號1樓,02-87712437,週一至週五08:00~17:00,101室,飲水服務;尿布台;母乳專用冰箱;免費衛生用品;置物空間;母乳哺育宣導資訊;設置意見回饋箱,無,使用者請於開放時間向本署秘書室事務課洽借鑰匙，並填寫哺(集)乳室鑰匙借用登記表、使用登記統計...,臺北市松山區八德路2段342號1樓,02-87712437,週一至週五08:00~17:00,松山區,使用者請於開放時間向本署秘書室事務課洽借鑰匙，並填寫哺(集)乳室鑰匙借用登記表、使用登記統計...,內政部營建署,101室,"[飲水服務, 尿布台, 母乳專用冰箱, 免費衛生用品, 置物空間]"
2,松山區,交通部運輸研究所,臺北市松山區敦化北路240號,02-23496734轉6733,每日24小時開放,地下1樓國際會議廳後方,飲水服務;母乳專用冰箱;置物空間;嬰兒車停放空間;母乳哺育宣導資訊,可提供輪椅媽媽使用,哺集乳室資訊依各場所實際公告內容為主,臺北市松山區敦化北路240號,02-23496734轉6733,每日24小時開放,松山區,哺集乳室資訊依各場所實際公告內容為主,交通部運輸研究所,地下1樓國際會議廳後方,"[輪椅, 飲水服務, 母乳專用冰箱, 置物空間, 嬰兒車停放空間]"
3,松山區,交通部公路總局臺北市區監理所,臺北市松山區八德路4段21號,02-27630155,週一至週五08:30~18:00,2樓電梯旁,尿布台;母乳專用冰箱;置物空間;嬰兒車停放空間;母乳哺育宣導資訊;進行滿意度調查,可提供輪椅媽媽使用,自行前往哺集乳室；哺集乳室資訊依各場所實際公告內容為主,臺北市松山區八德路4段21號,02-27630155,週一至週五08:30~18:00,松山區,自行前往哺集乳室；哺集乳室資訊依各場所實際公告內容為主,交通部公路總局臺北市區監理所,2樓電梯旁,"[輪椅, 尿布台, 母乳專用冰箱, 置物空間, 嬰兒車停放空間]"
4,松山區,臺北自來水事業處東區營業分處,臺北市松山區光復北路266號,02-87703362,週一至週五08:30~17:30,1樓3號服務櫃台正對面,飲水服務;尿布台;母乳專用冰箱;置物空間;嬰兒車停放空間;母乳哺育宣導資訊;進行滿意度調查,無,自行前往哺集乳室，並登記；哺集乳室資訊依各場所實際公告內容為主,臺北市松山區光復北路266號,02-87703362,週一至週五08:30~17:30,松山區,自行前往哺集乳室，並登記；哺集乳室資訊依各場所實際公告內容為主,臺北自來水事業處東區營業分處,1樓3號服務櫃台正對面,"[飲水服務, 尿布台, 母乳專用冰箱, 置物空間, 嬰兒車停放空間]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
533,北投區,臺北市政府消防局第四大隊陽明山分隊,臺北市北投區勝利路1號,02-28616119,每日24小時開放,1樓值班室旁,飲水服務;尿布台;置物空間;母乳哺育宣導資訊,可提供輪椅媽媽使用,自行前往哺集乳室，並登記；哺集乳室資訊依各場所實際公告內容為主,臺北市北投區勝利路1號,02-28616119,每日24小時開放,北投區,自行前往哺集乳室，並登記；哺集乳室資訊依各場所實際公告內容為主,臺北市政府消防局第四大隊陽明山分隊,1樓值班室旁,"[輪椅, 飲水服務, 尿布台, 置物空間]"
534,北投區,臺北市政府環境保護局北投垃圾焚化廠,臺北市北投區洲美街271號,02-28360050,週一至週五09:00~17:00,1樓主題館,以室內設備為主,以室內設備為主,自行前往哺集乳室，並登記；哺集乳室資訊依各場所實際公告內容為主,臺北市北投區洲美街271號,02-28360050,週一至週五09:00~17:00,北投區,自行前往哺集乳室，並登記；哺集乳室資訊依各場所實際公告內容為主,臺北市政府環境保護局北投垃圾焚化廠,1樓主題館,[無資料]
535,北投區,陽明山國家公園管理處陽明書屋,臺北市北投區中興路12號,02-28611444,週一至週五09:00~17:00,遊客中心,飲水服務;尿布台;母乳專用冰箱;免費衛生用品;置物空間;嬰兒車停放空間;母乳哺育宣導資訊;設...,可提供輪椅媽媽使用,自行前往哺集乳室，並登記；哺集乳室資訊依各場所實際公告內容為主,臺北市北投區中興路12號,02-28611444,週一至週五09:00~17:00,北投區,自行前往哺集乳室，並登記；哺集乳室資訊依各場所實際公告內容為主,陽明山國家公園管理處陽明書屋,遊客中心,"[輪椅, 飲水服務, 尿布台, 母乳專用冰箱, 免費衛生用品, 置物空間, 嬰兒車停放空間]"
536,北投區,家樂福北投店,臺北市北投區中和街366號,0963-079728,週一至週日08:00~23:00,地下1樓,以室內設備為主,以室內設備為主,自行前往哺集乳室，並登記；哺集乳室資訊依各場所實際公告內容為主,臺北市北投區中和街366號,0963-079728,週一至週日08:00~23:00,北投區,自行前往哺集乳室，並登記；哺集乳室資訊依各場所實際公告內容為主,家樂福北投店,地下1樓,[無資料]


In [56]:
breastfeeding_map = breastfeeding_map.iloc[:,[4,6,1,8,3,2,7,5,0]]

In [63]:
breastfeeding_map_gdf = gpd.GeoDataFrame(breastfeeding_map, crs="EPSG:4326")
geojson_dict = json.loads(breastfeeding_map_gdf.to_json())
# Remove the 'id' field
for feature in geojson_dict['features']:
    feature.pop('id', None)
geojson_dict['crs'] = { 
                       "type": "name", 
                       "properties": { 
                           "name": "urn:ogc:def:crs:OGC:1.3:CRS84" 
                           } 
                       }
# Write the dictionary to a GeoJSON file using json.dump
with open("4003.geojson", "w", encoding="utf-8") as f:
    json.dump(geojson_dict, f, ensure_ascii=False, indent=4)

In [46]:
breastfeeding_chart = breastfeeding_copy
services_list_select = ['輪椅','尿布台','置物空間', '嬰兒車停放空間']
services_dict_dist = {service: {dist: 0 for dist in specified_order } for service in services_list_select}

In [47]:
for i in range(len(breastfeeding_map["features"])):
    for service in breastfeeding_map.loc[i, "features"]:
        if(service in services_list_select):
            # print(service)
            # print(breastfeeding_map.loc[i, "dist"])
            services_dict_dist[service][breastfeeding_map.loc[i, "dist"]] += 1
services_dict_dist

{'輪椅': {'北投區': 26,
  '士林區': 27,
  '內湖區': 16,
  '南港區': 21,
  '松山區': 21,
  '信義區': 24,
  '中山區': 14,
  '大同區': 18,
  '中正區': 43,
  '萬華區': 21,
  '大安區': 20,
  '文山區': 29},
 '尿布台': {'北投區': 32,
  '士林區': 36,
  '內湖區': 19,
  '南港區': 27,
  '松山區': 31,
  '信義區': 38,
  '中山區': 23,
  '大同區': 27,
  '中正區': 58,
  '萬華區': 25,
  '大安區': 31,
  '文山區': 39},
 '置物空間': {'北投區': 31,
  '士林區': 34,
  '內湖區': 18,
  '南港區': 26,
  '松山區': 31,
  '信義區': 34,
  '中山區': 20,
  '大同區': 24,
  '中正區': 61,
  '萬華區': 26,
  '大安區': 28,
  '文山區': 42},
 '嬰兒車停放空間': {'北投區': 22,
  '士林區': 29,
  '內湖區': 13,
  '南港區': 22,
  '松山區': 26,
  '信義區': 24,
  '中山區': 13,
  '大同區': 21,
  '中正區': 49,
  '萬華區': 21,
  '大安區': 25,
  '文山區': 29}}

In [48]:
result = {
    "data":[
        {
            "name": service,
            "data":[
                services_dict_dist[service][dist] 
                for dist in specified_order
            ]
        }
        for service in services_list_select
    ]
}
result

{'data': [{'name': '輪椅',
   'data': [26, 27, 16, 21, 21, 24, 14, 18, 43, 21, 20, 29]},
  {'name': '尿布台', 'data': [32, 36, 19, 27, 31, 38, 23, 27, 58, 25, 31, 39]},
  {'name': '置物空間', 'data': [31, 34, 18, 26, 31, 34, 20, 24, 61, 26, 28, 42]},
  {'name': '嬰兒車停放空間',
   'data': [22, 29, 13, 22, 26, 24, 13, 21, 49, 21, 25, 29]}]}

In [49]:
with open('4003.json', "w", encoding='utf-8') as json_file:
    json.dump(result, json_file, ensure_ascii=False, indent=4)